In [3]:
%matplotlib inline

import numpy as np
import os,sys
from PIL import Image
from helpers import *
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from pylab import *

# Loading of the images 

In [4]:
# Loaded a set of images

#root_dir = "augmented_training/"
root_dir = "training/"
image_dir = root_dir + "images/"
files = os.listdir(image_dir)  

n = 100

print("Loading " + str(n) + " satellite images")
imgs = [load_image(image_dir + files[i]) for i in range(n)] #List of each image Dim (100, 400,400,3)

gt_dir = root_dir + "groundtruth/"
files2 = os.listdir(gt_dir)

print("Loading " + str(n) + " groundtruth images")
gt_imgs = [load_image(gt_dir + files2[i]) for i in range(n)]
print('Image size = ' + str(imgs[0].shape[0]) + ',' + str(imgs[0].shape[1]))

Loading 100 satellite images
Loading 100 groundtruth images
Image size = 400,400


# Extract patches from input images

In [5]:
patch_size = 16 # each patch is 16*16 pixels  #400/16 = 25*25 = 625 patches for 10 images

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]   # dim img_patches = (n,625,16,16,3)
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches

img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

print("img_patches:" + str(img_patches.shape))
print("gt_patches:" + str(gt_patches.shape))

img_patches:(62500, 16, 16, 3)
gt_patches:(62500, 16, 16)


# Building of the Dataset & Polynomial

In [6]:
# Print feature statistics

def build_X_Y_classes(img_patches,gt_patches,foreground_threshold):
    
    X = np.asarray([extract_features(img_patches[i]) for i in range(len(img_patches))])
    Y = np.asarray([value_to_class(np.mean(gt_patches[i]),foreground_threshold) for i in range(len(gt_patches))])

    #Y0 = [i for i, j in enumerate(Y) if j == 0] #index of Y where  y == 0 
    #Y1 = [i for i, j in enumerate(Y) if j == 1] #index of Y where y == 1
    
    return X,Y

def build_poly(X,n):
    
    poly = PolynomialFeatures(n, interaction_only=False) #Including interaction 
    return poly.fit_transform(X)


# Logistic regression 

In [9]:
from sklearn.metrics import f1_score

def logistic_regression_test(degree):
        foreground_threshold = 0.25
        t1 = datetime.datetime.now()
        X,Y = build_X_Y_classes(img_patches,gt_patches,foreground_threshold) 
        t2 = datetime.datetime.now()
        
        print("Build classes time: ",t2 - t1)
        
        tX = build_poly(X,degree)
        logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced",max_iter = 100)
    
        t3 = datetime.datetime.now()
        logreg.fit(tX,Y)
        t4 = datetime.datetime.now()
        
        print("Log_reg time: ", t4 - t3)
        
    # Predict on the training set
        Z = logreg.predict(tX)
    
    # Get non-zeros in prediction and grountruth arrays

        Zn = np.nonzero(Z)[0]
        Yn = np.nonzero(Y)[0]

        ratio = f1_score(Y,Z, average = 'macro')
        
        weight = logreg.coef_
        print('Ratio = ' + str(ratio))

        return ratio, weight 


In [13]:
ratios = []
weights = []
degrees = range(1,5)

for n in degrees:
    print("Degree {}\n".format(n))
          
    ratio, weight = logistic_regression_test(n)
          
    ratios.append(ratio)
    weights.append(weight)
    
print("Max ratio {a} at degree {b}".format(a = max(ratios) , b = ratios.index(max(ratios)))) 

      
    

Degree 1

Build classes time:  0:00:11.066700
Log_reg time:  0:00:00.176108
Ratio = 0.563834842574
Degree 2



KeyboardInterrupt: 

# Other stuff

In [ ]:
patch_size = 16

def extract_img_features(filename):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size)
    X = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))])
    return X

# Run prediction on the img_idx-th image
img_idx = 12

Xi = extract_img_features(image_dir + files[img_idx])
tXi = build_poly(Xi,3)
Zi = logreg.predict(tXi)
print(Zi.shape)
plt.scatter(Xi[:, 0], Xi[:, 4], c=Zi, edgecolors='k', cmap=plt.cm.Paired)

In [ ]:
w = gt_imgs[img_idx].shape[0]
h = gt_imgs[img_idx].shape[1]
predicted_im = label_to_img(w, h, patch_size, patch_size, Zi)
cimg = concatenate_images(imgs[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = make_img_overlay(imgs[img_idx], predicted_im)

plt.imshow(new_img)


In [ ]:
# Plot features using predictions to color datapoints
plt.scatter(X[:, 0], X[:, 4], c=Z, edgecolors='k', cmap=plt.cm.Paired)

In [ ]:

# Plot 2d features using groundtruth to color the datapoints

subplot(1,3,1)
subplots_adjust(left=None, bottom=None, right=2, top=None, wspace=None, hspace=None)
plt.scatter(X[:, 0], X[:, 3], c=Y, edgecolors='k', cmap=plt.cm.Paired) #c is the color  #orange dot = class 1 = foreground
plt.title("Red")
subplot(1,3,2)
plt.scatter(X[:, 1], X[:, 4], c=Y, edgecolors='k', cmap=plt.cm.Paired) #c is the color  #orange dot = class 1 = foreground
plt.title("Green")
subplot(1,3,3)
plt.scatter(X[:, 2], X[:, 5], c=Y, edgecolors='k', cmap=plt.cm.Paired) #c is the color  #orange dot = class 1 = foreground
plt.title("Blue")

In [ ]:
np.mean(gt_patches[4])

In [ ]:
extract_features(img_patches[4])

In [33]:
np.linspace(1,10,10)

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])

In [6]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'